# Detecting Occumotoric Events

Eye-tracking data are typically segmented into discrete events such as `fixations` and `saccades`. Fixations represent periods during which the eyes remain relatively stable, enabling visual information processing. Saccades are the rapid eye movements that shift gaze between fixations. Detecting these events and computing their properties, such as fixation duration and dispersion, or saccade amplitude and peak_velocity, forms the foundation for analyzing visual behavior and understanding how participants explore a stimulus.

For a detailed description of the algorithms and additional parameters, see the following tutorials: {doc}`Handling Gaze Events <../tutorials/event-handling>` and {doc}`Plotting Gaze Data <../tutorials/plotting>`.

## Fixations

Fixations can be detected using either the `I-VT` or the `I-DT` algorithm.

The `I-VT` (Velocity-Threshold Identification) method classifies each sample based on its velocity. Samples with velocities below a specified threshold are labeled as fixation points. Consecutive fixation samples are then merged into fixation events. A commonly used default threshold is 20 degrees per second, though this value may vary depending on the recording setup and research question.

The `I-DT` (Dispersion-Threshold Identification) method groups consecutive samples whose spatial dispersion remains below a predefined threshold and whose duration exceeds a minimum value. The algorithm slides a moving window over the data: if the dispersion within the window is sufficiently small, the window is classified as a fixation and is expanded until the dispersion criterion is violated.

`pymovements` implements these methods with the corresponding functions:
- I-VT: {py:func}`pymovements.events.detection.ivt`
- I-DT: {py:func}`pymovements.events.detection.idt`

In [ ]:
import pymovements as pm
from pymovements.gaze.experiment import Experiment

experiment = Experiment(
    screen_width_px=1680,
    screen_height_px=1050,
    screen_width_cm=47.5,
    screen_height_cm=30,
    distance_cm=65,
    origin='upper left',
    sampling_rate=1000.0,
)

gaze = pm.gaze.from_csv(
    "../examples/gaze-toy-example.csv",
    experiment=experiment,
    time_column="timestamp",
    pixel_columns=['x', 'y']
)

gaze.pix2deg()
gaze.pos2vel()

gaze.detect('ivt', name='fixation_ivt')

gaze.events.frame

## Saccades

Saccades are rapid eye movements that shift the point of fixation from one location to another. In `pymovements`, saccades (including microsaccades) can be detected from the velocity signal using the {py:func}`pymovements.events.detection.microsaccades` algorithm. This method implements a **noise-adaptive velocity threshold**. Instead of using a fixed velocity cutoff, the threshold is scaled relative to the noise level of the velocity signal. This makes the detection procedure more robust across recordings with different noise characteristics.

Two key parameters are necessary for the identification of saccades:

- **`threshold_factor`** controls how strict the velocity threshold is (default: `6`). Higher values detect fewer saccades (more conservative), lower values detect more (more sensitive).

- **`minimum_duration`** defines the minimum length of a velocity peak to be considered a saccade (default: `6` samples). Shorter events are treated as noise and ignored.

In [ ]:
gaze.detect('microsaccades', minimum_duration=6, threshold_factor=6)
gaze.events.frame

## The `Events` Object

Event detection in `pymovements` returns an {py:class}`~pymovements.Events` object. This object provides a structured representation of detected events and their properties. Each row in an `Events` object represents a single event and contains at least the following minimal schema:

- `name` – the event type (e.g., `"fixation_ivt"`, `"saccade"`)
- `onset` – event start time (in the same time unit as the gaze timestamps)
- `offset` – event end time
- `duration` – automatically computed as `offset - onset`

Additional event-specific properties (e.g., dispersion, amplitude, peak velocity) are stored as extra columns in the underlying `polars.DataFrame`.